In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
import os
import pandas as pd
from PIL import Image
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import joblib
import tensorflow as tf
from keras.regularizers import l1_l2
from keras.regularizers import l1
from keras.regularizers import l2
from tensorflow.keras.layers import BatchNormalization
import sklearn.metrics as metrics
import seaborn as sns
from datetime import datetime

from tensorflow.keras.callbacks import ReduceLROnPlateau

In [ ]:
# Daset OriginalFace
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
DATA_DIR1="/content/drive/My Drive/Face_recognition_project/data"

In [ ]:
# # To create rotated images in order to augment the data in the original dataset
# #Rotation de 45°
# import os
# import cv2
# import numpy as np

# def rotate_image(image, angle):
#     rows, cols = image.shape[0], image.shape[1]
#     M = cv2.getRotationMatrix2D((cols/2,rows/2),angle,1)
#     return cv2.warpAffine(image, M, (cols,rows))

# def rotate_and_save_images(folder_path, output_folder, angle):
#     if not os.path.exists(output_folder):
#         os.makedirs(output_folder)
#     for filename in os.listdir(folder_path):
#         image = cv2.imread(os.path.join(folder_path, filename))
#         rotated_image = rotate_image(image, angle)
#         cv2.imwrite(os.path.join(output_folder,filename+"rotated"+str(angle)), rotated_image)
        

# rotate_and_save_images(f"{DATA_DIR1}/Faces_removed_classes_28c",f"{DATA_DIR1}/Faces_removed_classes_28c_aug2", 5)
# print("ok")
# rotate_and_save_images(f"{DATA_DIR1}/Faces_removed_classes_28c",f"{DATA_DIR1}/Faces_removed_classes_28c_aug2", -5)
# print("ok2")

In [ ]:
# to get class names from Originale_face_dataset
img_files = os.listdir(f"{DATA_DIR1}/Faces_removed_classes_28c_aug")
y_img_labels = [file_name.split("_")[0] for file_name in img_files[:30]]
unique_y_img_labels = list(set(y_img_labels))
print(unique_y_img_labels)

# Puts a number on each class name in order
dict_labels = {}
for elt in unique_y_img_labels:
  dict_labels[elt] = unique_y_img_labels.index(elt)

print(dict_labels)
number_of_classes = len(dict_labels)
print("The number of classes is :", number_of_classes)

['Andy Samberg', 'Priyanka Chopra', 'Roger Federer', 'Camila Cabello', 'Hugh Jackman', 'Virat Kohli', 'Billie Eilish', 'Dwayne Johnson', 'Vijay Deverakonda', 'Anushka Sharma', 'Alia Bhatt', 'Margot Robbie', 'Tom Cruise', 'Zac Efron', 'Courtney Cox', 'Henry Cavill', 'Elizabeth Olsen', 'Amitabh Bachchan', 'Charlize Theron', 'Alexandra Daddario', 'Ellen Degeneres', 'Jessica Alba', 'Hrithik Roshan', 'Claire Holt', 'Robert Downey Jr', 'Lisa Kudrow', 'Brad Pitt', 'Natalie Portman']
{'Andy Samberg': 0, 'Priyanka Chopra': 1, 'Roger Federer': 2, 'Camila Cabello': 3, 'Hugh Jackman': 4, 'Virat Kohli': 5, 'Billie Eilish': 6, 'Dwayne Johnson': 7, 'Vijay Deverakonda': 8, 'Anushka Sharma': 9, 'Alia Bhatt': 10, 'Margot Robbie': 11, 'Tom Cruise': 12, 'Zac Efron': 13, 'Courtney Cox': 14, 'Henry Cavill': 15, 'Elizabeth Olsen': 16, 'Amitabh Bachchan': 17, 'Charlize Theron': 18, 'Alexandra Daddario': 19, 'Ellen Degeneres': 20, 'Jessica Alba': 21, 'Hrithik Roshan': 22, 'Claire Holt': 23, 'Robert Downey Jr':

In [ ]:
# now use the dict to construct digit labels according to each of the classes it belongs to 
# and stores it as a matrix (if i'm correct)
# according to the number of images

y_labels = np.array([dict_labels[elt] for elt in y_img_labels])
print(len(y_labels))

2450


In [ ]:
# Recupère les images sous forme de matrice
#X = np.array([np.array(Image.open(f"{DATA_DIR1}/Faces/{file_name}")) for file_name in img_files])
# print(X.shape)

(2450, 160, 160, 3)


In [ ]:
# # Saving array
# def save_array_to_file(array, filename):
#     with open(filename, 'wb') as f:
#         np.save(f, array)
# X_saved = save_array_to_file(X, '/content/drive/My Drive/Face_recognition_project/X_array_dataset1.npy')

In [ ]:
# !!!!!!!!!!!!!! Utilise trop de ram       To load array from saved array_file
def load_array_from_file(filename):
    with open(filename, 'rb') as f:
        array = np.load(f,allow_pickle=True)
        f.close()
    return array
X_loaded_array = load_array_from_file('/content/drive/My Drive/Face_recognition_project/X_array_dataset1.npy')

In [ ]:
print(y_labels.shape)
print(X_loaded_array.shape)

(2450,)
(2450, 160, 160, 3)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_loaded_array, y_labels, test_size=0.25, random_state=4)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1837, 160, 160, 3)
(613, 160, 160, 3)
(1837,)
(613,)


In [ ]:
from keras.preprocessing.image import ImageDataGenerator

# create an instance of the ImageDataGenerator class with desired augmentation parameters
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

# create augmented batches of the training data

train_generator = datagen.flow(X_train, y_train, batch_size=1837)
X_train_aug, y_train_aug = next(train_generator)

print(X_train_aug.shape)
print(y_train_aug.shape)

(1837, 160, 160, 3)
(1837,)


Normalize imgs

In [ ]:
X_train = X_train_aug.astype('float16')
X_test = X_test.astype('float16')

X_train /= 255
X_test /= 255

X_train = X_train.reshape(X_train.shape[0], 160, 160, 3)
X_test = X_test.reshape(X_test.shape[0], 160, 160, 3)

In [ ]:
import tensorflow as tf
from keras.applications import MobileNetV2,ResNet50,VGG16,VGG19
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout,GlobalAveragePooling2D,AveragePooling2D

im_shape = (160, 160, 3)

#VGG16  (meilleur modèle jusque la)
pretrained_model= tf.keras.applications.VGG16(include_top=False,
                   input_shape=im_shape,
                   pooling='avg',classes=number_of_classes,
                   weights='imagenet')

model = Sequential()
for each_layer in pretrained_model.layers:
        each_layer.trainable=True
model.add(pretrained_model)
#model.add(Flatten())
model.add(Dense(number_of_classes, activation='softmax'))

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=Adam(learning_rate=0.0001),
    metrics=['accuracy'],
)
model.summary()

58889256/58889256 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 512)               14714688  
                                                                 
 dense (Dense)               (None, 28)                14364     
                                                                 
Total params: 14,729,052
Trainable params: 14,729,052
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# im_shape = (160, 160, 3)

# cnn_model = Sequential()
# cnn_model.add(Conv2D(filters=36, kernel_size=7, activation='relu', input_shape= im_shape))
# cnn_model.add(MaxPooling2D(pool_size=2))
# cnn_model.add(Conv2D(filters=54, kernel_size=5, activation='relu'))
# cnn_model.add(MaxPooling2D(pool_size=2))
# cnn_model.add(Flatten())
# cnn_model.add(Dense(1024, activation='relu'))
# cnn_model.add(Dropout(0.3))
# cnn_model.add(Dense(512, activation='relu'))
# cnn_model.add(Dropout(0.3))
# cnn_model.add(Dense(256, activation='relu'))
# cnn_model.add(Dropout(0.3))
# # 31 is the number of classes (different people)  
# cnn_model.add(Dense(31, activation='softmax'))

# cnn_model.compile(
#     loss='sparse_categorical_crossentropy',#'categorical_crossentropy',
#     optimizer=Adam(learning_rate=0.0001),
#     metrics=['accuracy'],
# )
# cnn_model.summary()
# cnn_model.save('/content/drive/My Drive/Face_recognition_project/my_model_22.h5')
# saved_model2 = tf.keras.models.load_model('/content/drive/My Drive/Face_recognition_project/my_model_22.h5')
# from google.colab import files
# files.download('/content/drive/My Drive/Face_recognition_project/my_model_22.h5') 
# print(saved_model2)

In [ ]:
start_time = datetime.now()

reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.001, mode="auto", patience=1, min_lr=0.0001) # --> à mettre à 0.0001
history=model.fit(
                      np.array(X_train_aug), np.array(y_train_aug),
                      batch_size=256, # --> change to 256
                      epochs=50,
                      validation_data= (np.array(X_test), np.array(y_test)),
                      verbose=2,
                      #callbacks=[reduce_lr]
                      )
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

Epoch 1/50


In [ ]:
# Saving model
model.save('/content/drive/My Drive/Face_recognition_project/my_model_2.h5')

In [ ]:
saved_model = tf.keras.models.load_model('/content/drive/My Drive/Face_recognition_project/my_model_2.h5')

In [ ]:
# from google.colab import files
# files.download('/content/drive/My Drive/Face_recognition_project/my_model_2.h5')

In [ ]:
# Evaluate the model on the test data
train_loss, train_acc = saved_model.evaluate(X_train, y_train)
print('Train accuracy:', train_acc)

In [ ]:
# Evaluate the model on the test data
test_loss, test_acc = saved_model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)

In [ ]:
yhat = saved_model.predict(X_test)
predictions = [np.argmax(elt) for elt in yhat]
print(predictions)

In [ ]:
import matplotlib.pyplot as plt

# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
my_conf_matrix = confusion_matrix(y_test, predictions)

In [ ]:
import sklearn.metrics as metrics
import seaborn as sns
plt.figure(1, figsize=(15,5))
sns.heatmap(metrics.confusion_matrix(y_test, predictions))